[View in Colaboratory](https://colab.research.google.com/github/sungreong/TIL/blob/master/Decorator_python.ipynb)

In [0]:
def outer_function() : 
    print( "1. this is outer function!")
    def inner_function() :
        print( "2 . this is inner function , inside outer function!")
    print("3 . this is outside inner function inside outer function!")
    return inner_function()



In [4]:
func_assign = outer_function()


1. this is outer function!
3 . this is outside inner function inside outer function!
2 . this is inner function , inside outer function!


innef_function이 마지막으로 출력이 되는 것을 확인
1번 3번 진행 후 2번이 프린트가 된다  

Python decorator 는 

1 . value 값을 return 하는 것 처럼 다른 함수를 return 해주거나 2
. argument 처럼 함수를 받아준다.

In [10]:
import time

def timetest( input_func ) :
    
    def timed(*args , **kwargs ) :
        start_time = time.time()
        result = input_func( *args , **kwargs )
        end_time = time.time()
        
        print(" method name -{0} , args - {1}, Kwargs - {2} , Execution Time -{3}".format(
        input_func.__name__ , 
        args ,
        kwargs ,
        end_time - start_time )
             )
        return result
    return timed

@timetest
def foobar( *args , **kwargs ) :
    time.sleep(0.3)
    print(" inside foobar ")
    print( args , kwargs )
 

foobar(["hello , world",["foo"]], foo="1" , bar= 5  , bar2= "aa")

 inside foobar 
(['hello , world', ['foo']],) {'foo': '1', 'bar': 5, 'bar2': 'aa'}
 method name -foobar , args - (['hello , world', ['foo']],), Kwargs - {'foo': '1', 'bar': 5, 'bar2': 'aa'} , Execution Time -0.30121350288391113


@ 를 이용하여 decorator 를 호출하고 데코레이터 함수를 전달한다

Method Decorator

이것을 이용하면 class properties 를 decorating 으로써 overriding 해준다 따른 calling function 찾지 않아도!

In [24]:
def method_decorator(method):

    def inner(city_instance):
        if city_instance.name == "SFO":
            print("Its a cool place to live in.")
        else : 
            method(city_instance)
    return inner


class City(object):

    def __init__(self, name):
        self.name = name

    @method_decorator
    def print_test(self):
        print("1")
        print(self.name)

p1 = City("SFO")

p1.print_test()


Its a cool place to live in.


Class Decorators
* If you want to create a callable returning another callable, the function decorator approach is easier
* If you want the return to be a function, function decorators should be preferred, however if you want the decorator to return a custom object that does something different to what a function does, in that case a class decorator should be used.

In [43]:
class decoclass(object) : 
    
    def __init__(self , f ) : 
        self.f = f 
   
    def __call__(self , *args ,**kwargs ) :
        # before f actions
        print( "decorator initialised ")
        self.f(*args ,**kwargs ) 
        print("decorator terminated ")
        # after f actions
        
        
@decoclass 
def klass( *args  , **kwargs ) :
    print("decorator 초기와 끝남 사이에서 동작한다. " )
    s  = 0 
    for i in args : 
        s += i 
        
    for i , j in kwargs.items() :
        print( i , j )
    print("sum is" , s )

klass( 1, 3 , 5 , name="item" , sport ="ho~" )

decorator initialised 
decorator 초기와 끝남 사이에서 동작한다. 
name item
sport ho~
sum is 9
decorator terminated 


Chaining Decorators
* The chaining of decorator is similar to how multiple inheritance can be used to construct classes We can write as many decorator as we want and include them one by one in decoration line with decoration syntax before the definition of function to be decorated.

In [45]:
def makebold(f) :
    return lambda : "<b>" + f() + "</b>"

def makeitalic(f) :
    return lambda : "<i>" + f() + "</i>"

@makebold
@makeitalic
def say() :
    return "Hello"

print(say())

<b><i>Hello</i></b>


함수순서대로 감싸는 것 출력되는것 확인
* One thing should be kept in mind that the order of decorators we set matters. 
* When you chain decorators, the order in which they are stacked is bottom to top.

In [0]:
def decorator(func):
    #"""decorator docstring"""
    def inner_function(*args, **kwargs):
    #"""inner function docstring """
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return inner_function


@decorator
def foobar(x):
    #"""foobar docstring"""
    return x**2

In [63]:
print( foobar.__name__)
print( foobar.__doc__)
foobar(2)

inner_function
None
foobar was called


4

inner_function에 의해서 foobar가 대체되는 것을 확인했다.
* 이것은 통과할 때 함수의 정보를 잃는 것이라는 의미다


functional.wraps 는 이것을 해결해줄 수 있다.
*  It takes the function used in the decorator and adds the functionality of copying over the function name, docstring, arguemnets etc. 

Lets decorate without losing information:


In [58]:
from functools import wraps

def wrapped_decorator( func ) : 
    # wrapped decorator docstring
    @wraps(func)
    def inner_function( *args , **kwargs ) :
        ## inner function docstring
        print( func.__name__ + " was called ")
        return func(*args ,**kwargs)
    return inner_function

@wrapped_decorator
def foobar(x) :
    # foobar docstring
    return x**2



foobar


In [61]:
print(foobar.__name__)
print(foobar.__doc__)
print(foobar(2))

foobar
None
foobar was called 
4


## wraps 쓰고 안쓰고의 차이

* 복수의 데코레이터를 스택해서 사용하면 아래쪽 데코레이터부터 실행되는데, 위의 코드의 경우에는 #1의 my_logger가 먼저 실행되고 #2의 my_timer에게 #3에서 wrapper 함수를 인자로써 리턴하기 때문에 생기는 현상입니다.
* #4에서 original_function은 물론 wrapper 함수와 같습니다.

* 위와 같은 현상을 방지하기 위해서 만들어진 모듈이 있는데 그것이 functools 모듈의 wraps 데코레이터입니다

In [96]:
import datetime
import time


def my_logger(original_function):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_function.__name__), level=logging.INFO)
    
    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
        logging.info(
            '[{}] 실행결과 args - {}, kwargs - {}'.format(timestamp, args, kwargs))
        return original_function(*args, **kwargs)

    return wrapper  #3


def my_timer(original_function):  #4
    import time

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print('{} 함수가 실행된 총 시간: {} 초'.format(original_function.__name__, t2))
        return result

    return wrapper

 
@my_timer  #2
@my_logger #1
def display_info(name, age):
    time.sleep(1)
    print('display_info({}, {}) 함수가 실행됐습니다.'.format(name, age))

display_info('John', 25)

display_info(John, 25) 함수가 실행됐습니다.
wrapper 함수가 실행된 총 시간: 1.002373456954956 초


In [0]:
from functools import wraps
import datetime
import time


def my_logger(original_function):
    import logging
    logging.basicConfig(filename='{}.log'.format(original_function.__name__), level=logging.INFO)
    
    @wraps(original_function)  #1
    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
        logging.info(
            '[{}] 실행결과 args - {}, kwargs - {}'.format(timestamp, args, kwargs))
        return original_function(*args, **kwargs)

    return wrapper


def my_timer(original_function):
    import time

    @wraps(original_function)  #2
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print('{} 함수가 실행된 총 시간: {} 초'.format(original_function.__name__, t2))
        return result

    return wrapper





In [0]:
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print('display_info({}, {}) 함수가 실행됐습니다.'.format(name, age))



In [93]:
display_info('Jimmy', 30)

display_info(Jimmy, 30) 함수가 실행됐습니다.
display_info 함수가 실행된 총 시간: 1.0015435218811035 초
